In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
from joblib import dump

from src.scripts.artemis_data import get_ns_cols, get_email_cols

In [ ]:
file = "../data/processed/final_whois_data.csv"
df = pd.read_csv(file)


In [ ]:
label = 'malicious'
labels = df[label].values
data_df = df.drop(columns=label)

In [ ]:
data_df = data_df.drop(columns=get_ns_cols(7))
data_df = data_df.drop(columns=get_email_cols(3))
data_df = data_df.drop(columns=['domain', 'updated_date', 'expiration_date', 'creation_date', 'days_since_creation'])

In [ ]:
country_cols = ['country', 'dns_rec_a_cc', 'dns_rec_mx_cc']
countries_list = []
for col in country_cols:
    countries_list += data_df[col].tolist()
country_encoder = LabelEncoder().fit(countries_list)
for col in country_cols:
    data_df[col] = country_encoder.transform(data_df[col])

In [ ]:
cols_requiring_encoding = [
    'registrar', 'dnssec', 'org', 'state', 'whois_server', 'address', 'city', 'name',
    'zipcode', 'dns_rec_a_org', 'dns_rec_mx_org', 'main_name_server_domain', 'email_domains',
]
encoder_dict = {}
for col in cols_requiring_encoding:
    enc = LabelEncoder().fit(data_df[col])
    encoder_dict[col] = enc
    data_df[col] = enc.transform(data_df[col])


In [ ]:
data_df = data_df.fillna(-1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_df, labels, random_state=0)

In [ ]:
clf = RandomForestClassifier(criterion='entropy', random_state=0).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
model_loc = "../models/"
model_name = "rfc.joblib"
encoder_name = "enc_dict.joblib"
country_encoder_name = "country_encoder.joblib"

In [ ]:
dump(clf, model_loc+model_name)

In [ ]:
dump(encoder_dict, model_loc+encoder_name)

In [ ]:
dump(country_encoder, model_loc+country_encoder_name)